<a href="https://colab.research.google.com/github/AnnaKraim/ML/blob/main/simple_recommender_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Как связаны вектора и рекомендательные системы ?
#### Пишем маленький анализ.

Данный плейбук состоит из задач, связанных с применением векторов как инструмента рекомендательных систем.

Общая идея задачи - найти пользователей с близкими интересами.

Для вас собран тестовый датасет, где пользователь выражает свой интерес к той или иной вещи. 

Ваша задача в данной тетрадке найди похожих и не похожих пользователей. Что же, поехали!

### Задание 0. Чтение и просмотр данных.

В машинном обучении и Data Science данные принято хранить в виде таблицы объект, признак. 

Например, если есть два пользователя и нужно хранить их рост, пол, вес то данная таблица будет выглядить вот так:

Рост Вес Пол

160  55  0
180  85  1

Примером могут служить обычные excel таблицы.


---



Однако DataScienc'исты и программисты предпочитают формат **csv**: comma separated value. 

Это такие файлы, которые просто содержат список значений через запятую. На этом курсе мы часто будем работать именно с такими файлами используя различные инструменты, однако сегодня мы будем работать с ними вручную.



Давайте считаем файлы **user0.csv** и посмотрим что в нем находится

In [ ]:
lines = open('users0.csv').readlines()
lines[0:4]

['Имя,Отношение к алкоголю,Отношение к курению,Котики,Собачки,Реп-музыка,Рок-музыка,Читать,Кофе,Гулять\n',
 'Ваня,1,0,1,1,1,1,1,1,1\n',
 'Полина,1,0,1,1,1,1,1,1,1\n',
 'Коля,0,0,1,1,1,1,1,1,1\n']

Видно, что первая строка csv файлов обычно содержит поля, остальные строки содержат значение полей через запятую.

Чтобы избежать проблем с парсингом запятых и переносов строк, воспользуемся стандартной библиотекой python для чтения csv файлов.

In [ ]:
import csv
with open('users0.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        print(row[0], row[1:])

Имя ['Отношение к алкоголю', 'Отношение к курению', 'Котики', 'Собачки', 'Реп-музыка', 'Рок-музыка', 'Читать', 'Кофе', 'Гулять']
Ваня ['1', '0', '1', '1', '1', '1', '1', '1', '1']
Полина ['1', '0', '1', '1', '1', '1', '1', '1', '1']
Коля ['0', '0', '1', '1', '1', '1', '1', '1', '1']
Лиза ['-1', '-1', '1', '1', '1', '1', '1', '1', '1']
Рома ['1', '0', '1', '1', '0', '1', '1', '1', '0']
Катя ['1', '0', '1', '1', '1', '1', '1', '0', '1']
Саша ['-1', '1', '1', '1', '1', '1', '1', '1', '1']
Олеся ['-1', '-1', '1', '1', '-1', '1', '1', '1', '1']
Диана ['0', '-1', '1', '1', '1', '1', '1', '1', '1']
Нейтан ['1', '0', '1', '0', '1', '1', '0', '-1', '1']
Паша ['1', '1', '1', '1', '1', '1', '1', '1', '1']
Маша ['0', '1', '1', '0', '0', '1', '1', '-1', '-1']


Давайте напишем код, который считает всех пользователей в один массив, с которым нам будет удобно работать.

In [ ]:
import numpy as np
from collections import namedtuple

# Мы используем структуру namedtuple чтобы создать массив
# к которому можно обращаться как к классу.
User = namedtuple("User", ["Name", "Interest"])
users = []
with open('users0.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for i, row in enumerate(readCSV):
      # Пропустим строчку-заголовок
      if i == 0:
        continue
      name = row[0]
      # Считываем вектор интересов и приводим к типу одной строчкой
      interests = np.array(row[1:], dtype='int')
      users.append(User(name, interests))
print(users[0:2])
# Получить массив интересов 3 пользователя и его имя
print(users[2].Name, users[2].Interest)


[User(Name='Ваня', Interest=array([1, 0, 1, 1, 1, 1, 1, 1, 1])), User(Name='Полина', Interest=array([1, 0, 1, 1, 1, 1, 1, 1, 1]))]
Коля [0 0 1 1 1 1 1 1 1]


### Задание 1. Ищем бинарных похожих.

В файле users0.csv дана анкета для нескольких пользователей.

Анкета содержит несколько вопросов, на которые пользователь может ответить:

*   Отрицательно (-1)
*   Нейтрально (0)
*   Положительно (1)

Например, если в столбце **Котики** у пользователя стоит **1**, значит он положительно относится к котикам **^._.^**.

Используя векторы и зная, что все значения принимают 1, 0 и -1, мы можем попробовать найти похожих пользователей используя формулу **скалярного произведения**.

Идея простая: возьмем для примера вектор интереса пользователя Петя: Он любит котиков, и любит игры, но не любит собачек.

Вектор Пети будет (1,1,-1). 

У него есть подружка Маша, она тоже любит котиков, тоже любит игры, нейтрально относится к собачкам.

Вектор Маши будет (1,1,0)

Если мы перемножим вектор Пети на вектор Маши, мы получим 

$ 1 \cdot 1 + 1 \cdot 1 + -1 \cdot 0 = 2$

Если мы возьмем пример человека, который не любит котиков, не люит игры, но любит собачек и умножим его на вектор Пети, то получим 

$ 1 \cdot -1 + 1 \cdot -1 + -1 \cdot 1 = -3$

-3 меньше 2, очевидно что Маша ближе к Пете чем человек, который имеет абсолютно другие интересы.





#### Задание 1.1 

* Прочитайте файл **users0.csv** в массив **users** как показано в Задании 0

* Попробуйте посчитать скалярное произведение для первого и второго пользователя (Ваня и Полина). Какое получилось произведение ? Что можно сказать про интересы данных пользователей ?

In [ ]:
def sp(us1,us2):
  return us1.Interest.dot(us2.Interest)
print(sp(users[0],users[1]),"Интересы сильно совпадают")

8 Интересы сильно совпадают


#### Задание 1.2

* Какой пользователь наиболее близок Коле ?

* Какой пользователь наиболее близок Олесе ?

* Какой пользователь наиболее далек от Нейтана ?

Для каждого вопроса дайте ответ в виде пары пользователь, скалярное произведение



In [ ]:
s=""
max=[-10]*3
max[2]=10
for j in (2,7,9):
  for i in range(12):
    if i!=j and j!=9 and sp(users[j],users[i])>max[j//4]:
      max[j//4]=sp(users[j],users[i])
      s=users[i].Name
    if i!=j and j==9 and sp(users[j],users[i])<max[j//4]:
      max[j//4]=sp(users[j],users[i])
      s=users[i].Name
  print(s,max[j//4])

Ваня 7
Лиза 7
Олеся 0


#### Задание 1.3

Для каждого пользователя выдайте двух наиболее близких к нему других пользователей.



In [ ]:
i1=-1
i2=-1
max1=[-10]*12
max2=[-10]*12
for j in range(12):
  for i in range(12):
    if i!=j and sp(users[j],users[i])>max1[j]:
      max1[j]=sp(users[j],users[i])
      i1=i
  for i in range(12):
    if i!=j and sp(users[j],users[i])>max2[j] and i!=i1:
      max2[j]=sp(users[j],users[i])
      i2=i
  
  print(users[i1].Name,max1[j],users[i2].Name,max2[j])

Полина 8 Паша 8
Ваня 8 Паша 8
Ваня 7 Полина 7
Диана 8 Коля 7
Ваня 6 Полина 6
Ваня 7 Полина 7
Коля 7 Лиза 7
Лиза 7 Диана 6
Лиза 8 Ваня 7
Катя 5 Ваня 4
Ваня 8 Полина 8
Рома 2 Катя 2


### Задание 2. Добавляем чиселки.


Очевидно, что в реальных данных мы чаще всего имеем отношение с числовыми значениями. 

В файле **users1.csv** содержатся те же пользователи, но оценки теперь представлены в виде чисел от 1 до 10 включительно.

Файлы **users0.csv** и **users1.csv** соотносятся как, если оценка по 10 бальной была меньше 4, то ставим -1, если больше 6, то ставим 1, остальное - 0

Можно ли использовать формулу скалярного произведения для векторов с числовыми значениями ? 


#### Задание 2.1 Считываем опять.

* Считайте данные из файла user1.csv
* Попробуйте посчитать скалярное произведение между первым и всеми остальными пользователями.
* Сравните наиболее подходящего пользователя с результатом задания 1.3. Почему так получилось ?




In [ ]:
User = namedtuple("User", ["Name", "Interest"])
users = []
with open('users1.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for i, row in enumerate(readCSV):
      # Пропустим строчку-заголовок
      if i == 0:
        continue
      name = row[0]
      # Считываем вектор интересов и приводим к типу одной строчкой
      interests = np.array(row[1:], dtype='int')
      users.append(User(name, interests))
for i in range(1,12):
  print(users[i].Name,sp(users[0],users[i]))



Полина 440
Коля 437
Лиза 402
Рома 355
Катя 366
Саша 403
Олеся 377
Диана 363
Нейтан 355
Паша 440
Маша 381


#### Задание 2.2 Косинусное расстояние

В данном задании вам опять нужно будет находить похожих пользователей, однако теперь мы будем использовать косинусное расстояние для определения "похожести" двух векторов.

**Косинусная мера** - мера сходства между двумя векторами, которая высчитывает косинус угла между двумя векторами.

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/2a8c50526e2cc7aa837477be87eff1ea703f9dec)

[ссылка](https://wikimedia.org/api/rest_v1/media/math/render/svg/2a8c50526e2cc7aa837477be87eff1ea703f9dec)

#### Реализуйте функцию вычисления косинусного расстояния между двумя векторами используя numpy.

In [ ]:
import math
import numpy as np
def cos_distance(a,b):
  s=a.dot(b)
  dl=(math.sqrt(sum(a*a))*math.sqrt(sum(b*b)))
  c=s/dl
  return c

In [ ]:
# Для проверки себя:

a = np.array([1,2,3,4])
b = np.array([1,2,3,3])
round(cos_distance(a,b), 3) == 0.99

True

#### Задание 2.3 Ищем похожих

С помощью функции косинусного расстояния, найдите: 

*   Наиболее похожего для Дианы
*   Наиболее непохожего для Нэйтана




In [ ]:
s=""
max=[-10]*2
max[1]=10
j=8
for i in range(12):
  if i!=j and cos_distance(users[j].Interest,users[i].Interest)>max[j//9]:
    max[j//9]=cos_distance(users[j].Interest,users[i].Interest)
    s=users[i].Name
print(s,max[j//9])
j=9
for i in range(12):
  if i!=j and cos_distance(users[j].Interest,users[i].Interest)<max[j//9]:
    max[j//9]=cos_distance(users[j].Interest,users[i].Interest)
    s=users[i].Name
print(s,max[j//9])

Ваня 0.9913815228185777
Олеся 0.8384767138930074


#### Задание 2.4 Ищем похожих
С помощью функции косинусного расстояния, найдите для каждого пользователя найдите двух наиболее похожих на него.

Сравните с результами из задания 1. 

Есть ли совпадения ? Интерпретируйте результат

In [ ]:
i1=-1
i2=-1
max1=[-10]*12
max2=[-10]*12
for j in range(12):
  for i in range(12):
    if i!=j and cos_distance(users[j].Interest,users[i].Interest)>max1[j]:
      max1[j]=cos_distance(users[j].Interest,users[i].Interest)
      i1=i
  for i in range(12):
    if i!=j and cos_distance(users[j].Interest,users[i].Interest)>max2[j] and i!=i1:
      max2[j]=cos_distance(users[j].Interest,users[i].Interest)
      i2=i
  
  print(users[i1].Name,round(max1[j],3),users[i2].Name,round(max2[j],3))
print("Иногда встречаются совпадения, но редко")

Диана 0.991 Коля 0.989
Катя 0.983 Саша 0.976
Диана 0.991 Ваня 0.989
Коля 0.984 Олеся 0.973
Коля 0.977 Ваня 0.976
Полина 0.983 Коля 0.977
Ваня 0.987 Коля 0.979
Лиза 0.973 Коля 0.95
Ваня 0.991 Коля 0.991
Маша 0.97 Катя 0.968
Ваня 0.987 Саша 0.977
Катя 0.972 Нейтан 0.97
Иногда встречаются совпадения, но редко


### Задание 3. Добавляем возраст.

В файле **users_age.csv** в первой колонке записан возраст пользователя.

Будем считать, что мы также хотим учитывать возраст как один из параметров нашей модели и считать двух пользователей более похожими, если они примерно одного возраста.


#### Задание 3.1

Считайте данные из users_age.csv и повторите задание 2.4. 

Сильно ли изменились результаты ? 



In [ ]:
User = namedtuple("User", ["Name", "Interest"])
users = []
with open('users_with_age.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for i, row in enumerate(readCSV):
      # Пропустим строчку-заголовок
      if i == 0:
        continue
      name = row[0]
      # Считываем вектор интересов и приводим к типу одной строчкой
      interests = np.array(row[1:], dtype='int')
      users.append(User(name, interests))
i1=-1
i2=-1
max1=[-10]*12
max2=[-10]*12
for j in range(12):
  for i in range(12):
    if i!=j and cos_distance(users[j].Interest,users[i].Interest)>max1[j]:
      max1[j]=cos_distance(users[j].Interest,users[i].Interest)
      i1=i
  for i in range(12):
    if i!=j and cos_distance(users[j].Interest,users[i].Interest)>max2[j] and i!=i1:
      max2[j]=cos_distance(users[j].Interest,users[i].Interest)
      i2=i
  
  print(users[i1].Name,round(max1[j],3),users[i2].Name,round(max2[j],3))
print("У некоторых пользователей результаты изменились сильно")

Коля 0.995 Саша 0.99
Коля 0.987 Саша 0.986
Ваня 0.995 Лиза 0.992
Коля 0.992 Олеся 0.984
Катя 0.989 Паша 0.987
Маша 0.991 Нейтан 0.989
Ваня 0.99 Коля 0.987
Лиза 0.984 Коля 0.974
Ваня 0.989 Паша 0.989
Катя 0.989 Маша 0.989
Диана 0.989 Рома 0.987
Катя 0.991 Нейтан 0.989
У некоторых пользователей результаты изменились сильно


#### Задание 3.1


Возраст, в отличие от остальных факторов измеряется от 18 до 31 в данном датасете, нужно ли как-то масштабировать этот признак ?

Попробуйте отмасштабировать признак возраст:

Пусть возраст 
* 18 - 1
* 31 - 10

**Посчитайте косинусное расстояние между Колей и Ваней до масштабирования возраста и после**

Изменилось ли оно ?


In [ ]:
print(cos_distance(users[0].Interest,users[2].Interest))
for i in range(12):
  users[i].Interest[0]=1+(users[i].Interest[0]-18)*(9/13)
print(cos_distance(users[0].Interest,users[2].Interest))
print("Да, изменилось")

0.994624826177789
0.9893659917194179
Да, изменилось
